# ShopAssist AI 2.0

## Part 1: Introduction


#### Project Background

In today's digital age, online shopping has become the go-to option for many consumers. However, the overwhelming number of choices and the lack of personalized assistance can make the shopping experience daunting. To address this, we have developed **ShopAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given a dataset containing information about laptops (product names, specifications, descriptions, etc.), build a chatbot that parses the dataset and provides accurate laptop recommendations based on user requirements*.


You can load the data and see it here.

In [1]:
# Install OpenAI library
#!pip install -U -q openai tenacity

#### Import relevant libraries

In [2]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)

#### Read the dataset

In [3]:
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('laptop_data.csv')
df.head(5)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


#### Approach (using Open AI with Function API Calling):

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play (function call will be choosen using Functions API with Open AI), extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

## Part 2: System Design


#### Dataset

We have a dataset `laptop.csv` where  each row describes the features of a single laptop and also has a small description at the end. The chatbot that we build will leverage LLMs to parse this `Description` column and provide recommendations

#### Building the Chatbot

Below is the system design for the chatbot, we have seen so far.

![Chatbot_sys_design.png](Chatbot_sys_design1.png)

`Stage 1`

- Intent Clarity Layer

`Stage 2`

- Product Mapping Layer
- Product Information Extraction Layer

`Stage 3`

- Product Recommendation Layer


Note: We will not need Intent Confirmation Layer as we are using Function Api calling to capture the data

##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

Note: Because now we are using Function API calls, intent_confirmation_layer and dictionary_present steps can be dropped. Function API call will help in structuring the output of the layers are now output is more consistent and therefore, require less manipulation.

In the next sections, we will look at how to write the code for the above functions.

## Part 3: Implementation

## Stage 1 - Enhanced with Function API Calling

[Stage 1 Flowchart](https://cdn.upgrad.com/uploads/production/ed8cf454-0c7c-4c5d-bc05-23bb3cea6c57/image+3.jpg)

![Stage 1 Flowchart](Stage1.png)

### 3.1 - Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast

Make sure the api key is stored in the text file `OPENAI_API_Key.txt`.

In [4]:
## Import the necessary libraries for building the project
import os,json,ast
import openai
from openai import OpenAI
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

In [5]:
## Add your OPENAI API KEY
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

### 3.2 - Implementing Initialization and Intent Clarity

As mentioned earlier, this layer helps in identifying the user requirements and passing it on to the product matching layer. Here are the functions that we would be using for building these layers:

- `initialize_conversation()`


### `initialize_conversation()`:
This initializes the variable conversation with the system message. Using prompt engineering and chain of thought reasoning, the function will enable the chatbot to keep asking questions until the user requirements have been captured in a dictionary. It also includes Few Shot Prompting(sample conversation between the user and assistant) to align the model about user and assistant responses at each step.

In [6]:
def initialize_conversation():
    delimiter = "####"

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You are trying to understand the user's requirement for these laptop's features: ('gpu intensity','display quality','portability','multitasking','processing speed','budget')
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    After understanding their requirements, you'll use a function call to suggest the top 3 laptops with their respective user match score.
    Recommend these laptops and answer any user's query about them.

    {delimiter} Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    If the user's budget is less than says less than 25000 INR, please mention that there are no laptops in that range.
    Recommend the top3 laptops in the following format:
    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    3. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    {delimiter}

    {delimiter}To find the top3 laptops, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.{delimiter}
    {delimiter}

    {delimiter} Here is a sample conversation between the user and assistant:
    Assistant: "Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?"
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    {delimiter}


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

Let's now look at the next function.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant. We'll use the Chat Completions function for performing LLM calls to OpenAI.

### `get_chat_completions()`:

This function perform LLM call using the Chat Completions API to get the LLM response.

In [7]:
## Function Description for the Function Calling API (This will eliminate the need for intent clarification and dictionary present functions)
tools = [
            {
                "type": "function",
                "function": {
                    "name": "compare_laptops_with_user",
                    "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "gpu intensity": {
                                "type": "string",
                                "description": "The requirement of the user in GPU capacity classfied as low, medium or high" ,
                            },
                            "display quality": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high" ,
                            },
                            "portability": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's portability classfied as low, medium or high" ,
                            },
                            "multitasking": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high" ,
                            },
                            "processing speed": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high" ,
                            },
                            "budget": {
                                "type": "integer",
                                "description": "The maximum budget of the user" ,
                            },

                        },
                        "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"],
                    },
                }
            }
        ]

In [8]:
# Define a Chat Completions API call
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def get_chat_completions(messages,tools=None, tool_choice=None, model="gpt-4o", json_format = False):
    """
    Generate chat completions using OpenAI's GPT-3.5-turbo or 4o model.

    Parameters:
    - input: list of dictionaries representing chat messages to prompt the model.
    - json_format: boolean, specifies whether the output should be in JSON format (default: False).

    Returns:
    - output: The generated chat completion response. If json_format is True, returns a JSON object.

    """
    client = OpenAI()
    try:
        if json_format:
            response = client.chat.completions.create(
                model=model,
                max_tokens=500,
                response_format={ "type": "json_object" },
                messages=messages,
                temperature=0,
                tools=tools
            )
        else:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0, # this is the degree of randomness of the model's output
                max_tokens = 500,
                tools=tools
            )
        response = json.loads(response.to_json())
        # print(response)
        return response["choices"][0]["message"]
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

Typically, whenever the chatbot is interacting with the user, all the conversations should be moderated to identify any inappropriate content. Let's look at the function that can help with it.

### `moderation_check()`:
 This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [9]:
# Define a function called moderation_check that takes user_input as a parameter.
def moderation_check(user_input):
    """
    Perform moderation check on user input using the OpenAI API.

    Parameters:
    - user_input (str): The text input to be checked for moderation.

    Returns:
    - str: "Flagged" if the input is flagged by the moderation system,
           "Not Flagged" otherwise.
    """
    # Call the OpenAI API to perform moderation on the user's input.

    # Extract the moderation result from the API response.
    client = openai.OpenAI(
        api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
    )
    
    # Check if the input was flagged by the moderation system. If flagged, return "Flagged" else return "Not Flagged"
    response = client.moderations.create(input=user_input)
    response_dict = response.model_dump()
    moderation_output = response_dict["results"][0]
    if moderation_output["flagged"] == True:
        return "Flagged"
    else:
        return "Not Flagged"

## Stage 2

![Stage 2 Flowchart](Stage2.png)

### 3.3 Implementing the Product Mapping and Information Extraction Layers

In this section, we take in the output of the previous layers, i.e. the user requirements, which is in the format of a Python dictionary.
<br>
Next we will extract the top 3 laptop recommendations based on user's requirements.

This stage consists of the following helper functions that will implement the information extraction and product matching layers.

### `product_map_layer()` - To prepare dataset for search:

This function is responsible for extracting key features and criteria from laptop descriptions. Here's a breakdown of how it works:

-  Use a prompt that assign it the role of a Laptop Specifications Classifier, whose objective is to extract key features and classify them based on laptop descriptions.

- Provide step-by-step instructions for extracting laptop features from description.

- Assign specific rules for each feature (e.g., GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed) and associate them with the appropriate classification value (Low, Medium, or High).

- Includes Few Shot Prompting (sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

In [10]:
def product_map_layer(laptop_description):
    """
    Extracts key features from a laptop description and classifies them based on predefined rules.

    Args:
        laptop_description (str): The description of the laptop to be analyzed.

    Returns:
        dict: A dictionary containing the classified features of the laptop.
            Keys represent different aspects such as GPU intensity, Display quality, Portability,
            Multitasking, and Processing speed, each mapped to a value indicating the classification
            (low, medium, or high) based on the extracted features.
    """

In [11]:
def product_map_layer(laptop_description):

    delimiter = "#####"
    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Display Type, Screen Resolution, Display Size)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM Size)",
        "Processing speed":"(CPU Type, Core, Clock Speed)"
    }

    values = {'low','medium','high'}

    prompt=f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity: low: < for Integrated graphics or entry-level dedicated graphics like Intel UHD > , \n
    medium: < if Mid-range dedicated graphics like M1, AMD Radeon, Intel Iris > , \n
    high: < High-end dedicated graphics like Nvidia > , \n

    Display Quality: low: < if resolution below Full HD (e.g., 1366x768). > , \n
    medium: < if Full HD resolution (1920x1080) or higher. > , \n
    high: < if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. > \n

    Portability: low: < if laptop weight is less than 1.51 kg > , \n
    medium: < if laptop weight is between 1.51 kg and 2.51 kg> , \n
    high: <if laptop weight is greater than 2.51 kg> \n

    Multitasking: low: < If RAM size is 8GB, 12GB > , \n
    medium: < if RAM size is 16GB > , \n
    high: < if RAM size is 32GB, 64GB> \n

    Processing Speed: low: < if entry-level processors like Intel Core i3, AMD Ryzen 3> , \n
    medium: < if Mid-range processors like Intel Core i5, AMD Ryzen 5 > , \n
    high: < if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher > \n
    {delimiter}

    {delimiter}
    Here are some input output pairs for few-shot learning:
    input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for multitasking and productivity. With 16GB of RAM and an SSD, it offers fast and efficient performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for decent graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. Priced at 130,000, it offers top-notch performance and portability for professionals."
    output2: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'high', 'Multitasking':'high', 'Processing speed':'high'}}

    input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an Intel Core i9 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an AMD Radeon graphics card, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it is relatively lightweight for its size. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals. Priced at 280,000, it caters to users who require uncompromising performance and a superior display for their demanding workloads."
    output3: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'medium','Multitasking': 'high', 'Processing speed': 'high'}}
    {delimiter}
    """
    input = f"""Follow the prompt instructions step-by-step and output the dictionary in JSON format for the following laptop {laptop_description}."""

    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]
    response = get_chat_completions(messages, json_format=True)["content"]
    return response

Let's now apply this function to the entire laptop dataset

In [12]:
##Run this code once to extract product info in the form of a dictionary
laptop_df= pd.read_csv('laptop_data.csv')

## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))

In [13]:
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

### extract_dictionary_from_string()

In [14]:
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

### `compare_laptops_with_user()`:

This function compares the user's profile with the different laptops and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the laptops based on their price, keeping only the ones within the user's budget.
    - Calculate a score for each laptop based on how well it matches the user's requirements.
    - Sort the laptops based on their scores in descending order.
    - Return the top 3 laptops as a JSON-formatted string.

In [15]:
def compare_laptops_with_user(user_req_string):
    """
    Compares laptops based on user requirements and returns the top 3 matching laptops in JSON format.

    Parameters:
    - user_req_string (str): A string representing user requirements in JSON format.

    Returns:
    - str: A JSON string containing information about the top 3 matching laptops based on user requirements.
    """
    budget = user_req_string.get('budget', '0')
    laptop_df= pd.read_csv('updated_laptop.csv')
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    mappings = {
            'low': 0,
            'medium': 1,
            'high': 2
        }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in user_req_string.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
            ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1

        filtered_laptops.loc[index, 'Score'] = score

    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')

### `product_validation_layer()`:

This function verifies that the laptop recommendations are good enough, has score greater than 2, and matches the user's requirements.

In [16]:
def recommendation_validation(laptop_recommendation):
    """
    Validate a list of laptop recommendations based on a score threshold.

    Parameters:
    - laptop_recommendation (str): JSON string containing a list of laptop recommendations,
      each with a 'Score' key representing its rating.

    Returns:
    - list: A filtered list of laptop recommendations where each item has a 'Score' greater than 2.
    """
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])
    return json.dumps(data1)

Now that you the top 3 laptops extracted, let's pass it to the recommendation layer that'll send it to the user and the user can ask questions around it.

## Stage 3

![Stage 3 Flowchart](Stage3.png)

### 3.4: Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_laptops_with_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. Initialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.



In [23]:
def initialize_conv_reco(products):
    """
    Initializes a conversation recommendation system for a laptop gadget expert.

    Parameters:
    - products (list): A list of products to be included in the user's profile.

    Returns:
    - conversation (list): A list containing initial system and user messages for the conversation.

    Description:
    This function sets up a conversation recommendation system for an intelligent laptop gadget expert.
    The system message provides guidance on how to respond to user queries based on the product catalog.
    It instructs to summarize each laptop's major specifications and price, starting with the most expensive.
    The user message confirms the list of products included in the user's profile.

    Example:
    >>> products = ['Laptop A', 'Laptop B', 'Laptop C']
    >>> initialize_conv_reco(products)
    [{'role': 'system', 'content': 'You are an intelligent laptop gadget expert and you are tasked with the objective to solve the user queries about any product from the catalogue in the user message. You should keep the user profile in mind while answering the questions.\n\nStart with a brief summary of each laptop in the following format, in decreasing order of price of laptops:\n1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n\n'},
    {'role': 'user', 'content': " These are the user's products: ['Laptop A', 'Laptop B', 'Laptop C']"}]
    """
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content": user_message}]
    return conversation



## Combining all the 3 stages

In this layer, we combine all the three stages that we defined above.

`Stage 1` + `Stage 2` + `Stage 3`

### 3.5 Dialogue Management System

Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [24]:
def validate_user_input(user_input):
    # Moderation Check for the user input
    moderation = moderation_check(user_input)
    if moderation == 'Flagged':
      display("Sorry, this message has been flagged. Please restart your conversation.")
      return 0
    return 1

In [27]:
def dialogue_mgmt_system():
    """
    Manages a conversational system between a user and an assistant to recommend laptops.

    This function initializes the conversation, retrieves the introduction, handles user input,
    performs moderation checks, and manages the recommendation process based on user interactions.

    Returns:
        None
    """

    # Initialize the conversation and create the assistant object instance
    conversation = initialize_conversation()
    introduction = get_chat_completions(conversation)["content"]

    print(introduction + '\n')
    user_input = ""
    
    while(user_input != "exit"):
        # Get user input
        user_input = input("")
        # Moderation Check for the user input
        if validate_user_input(user_input) == 0:
            break
        # Add the user input to the conversation object
        conversation.append({"role": "user", "content": user_input})
        
        # Step 1: Pass the user input to GPT
        response_assistant = get_chat_completions(conversation,tools=tools)
        # Moderation Check for the assistant response
        if validate_user_input(response_assistant["content"]) == 0:
            break

        if "tool_calls" in list(response_assistant.keys()) and response_assistant["tool_calls"]:
            tool_called_message = "\nThank you for providing all the information. Kindly wait, while I fetch the products using function API call\n"
            conversation.append({"role": "assistant", "content": tool_called_message})
            print(tool_called_message)

            # Step 2: Extract top3 laptops by calling the external function
            tool  = response_assistant["tool_calls"][0]
            tool_call_id = tool["id"]
            function_name = tool["function"]["name"]
            function_args = json.loads(tool["function"]["arguments"])
            top_3_laptops = compare_laptops_with_user(function_args)
            # Validate the score of laptops found by matching the user requirements
            function_response = recommendation_validation(top_3_laptops)
            # Check if the function response is empty
            if len(function_response) == 0:
              print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
              break

            # Step 3: send the info on the function call and function response to GPT
            conversation.append(response_assistant)
            conversation.append(
                {
                    "tool_call_id": tool_call_id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

            # Format the recommendation response
            conversation_reco = initialize_conv_reco(function_response)
            formatted_recommendation = get_chat_completions(conversation_reco)
            recommendation = formatted_recommendation["content"]
            # Moderation Check for the assistant response
            if validate_user_input(recommendation) == 0:
                break
            
            conversation.append({"role": "assistant", "content": recommendation})
            print("\n" +recommendation + "\n")

            print("Enter exit to end the conversation or press enter to continue\n")
        else:
            conversation.append({"role": "assistant", "content": response_assistant["content"]})
            print("\n" +  response_assistant["content"] + "\n")
    
    # Save conversation in a text file
    with open("conversation.txt", "w") as file:
        for message in conversation:
            file.write(f"{message['role']}: {message['content']}\n")

In [28]:
dialogue_mgmt_system()

Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements or tell me a bit about how you plan to use the laptop?


Hello! Thank you for sharing your requirements. To ensure I have a complete understanding, I have a couple of questions:

1. Do you require a dedicated GPU for your software development tasks, or is an integrated GPU sufficient?
2. Do you often work with multiple virtual machines or heavy IDEs that might require more RAM?

Your answers will help me tailor the recommendations more accurately.


Thank you for providing all the information. Kindly wait, while I fetch the products using function API call


Here is a brief summary of each laptop in decreasing order of price:

1. **ASUS ROG Strix G**: Intel Core i7 (2.9 GHz), 16GB RAM, SSD, 17.3" IPS display (1920x1080), NVIDIA RTX graphics, Windows 10, 2.9 kg, Aura Sync RGB Keyboard, 2-year warranty, 5-hour battery life, Rs 85,000. A high-performance gaming laptop with c